In [10]:
from pymongo import MongoClient
import certifi
import os
from dotenv import load_dotenv
from utils.translation import translate_text
from utils.courses import find_courses_by_name, retrieve_lessons_text
import pandas as pd

# Setup

In [11]:
# Loads the path for GCP translation API and env vars
load_dotenv()

True

In [12]:
MONGO_URI = os.environ["MONGO_DB_URI"]
PATH = os.environ["DIR_PATH"]

In [13]:
#Load the gcf data from the MongoDB db
ca = certifi.where()
cluster = MongoClient(host=MONGO_URI, tlsCAFile=ca)
db = cluster["gcfglobal"]
collection = db["lesson"]
collection_tutorials = db['tutorial']

# Traducir las lecciones por curso

In [15]:
paths = [
    ('como-usar-o-twitter', 'como-usar-twitter'),
    ('excel-2016',), 
    ('microsoft-excel-2010',), 
    ('microsoft-word-2010',), 
    ('powerpoint-2019',), 
    ('word-2016',),
    ('algebra',), 
    ('credito',), 
    ('curso-de-fotografia-digital',), 
    ('curso-de-photoshop',), 
    ('informatica-basica',),
    ('whatsapp-business',), 
    ('virus-e-antivirus','virus-informaticos-y-antivirus'),
    ('criacao-de-sites','creacion-de-sitios-web'),
    ('seguranca-na-internet', 'seguridad-en-internet'),
    ('criar-um-correio-eletronico', 'crear-un-correo-electronico'),
    ('como-criar-um-blog', 'crear-un-blog-en-internet'),
    ('apresentacoes-basicas', 'presentaciones-basicas'),
    ('como-usar-o-telegram', 'curso-de-telegram'),
    ('tudo-sobre-o-windows-10', 'como-usar-windows-10'),
    ('como-usar-o-sistema-android', 'como-usar-android'),
    ('criar-uma-conta-no-facebook', 'crear-cuenta-en-facebook'),
    ('aplicativos', 'aplicaciones'),
    ('google-sala-de-aula-para-alunos', 'google-classroom-para-estudiantes'),
    ('somar-e-subtrair', 'sumar-y-restar'),
    ('como-fazer-multiplicacao', 'multiplicar'),
    ('divisao', 'dividir'),
    ('multiplos-e-divisores', 'divisores-y-multiplos'),
    ('numeros-fracionarios', 'fraccionarios'),
    ('os-numeros-decimais', 'los-decimales'),
    ('os-conjuntos-matematicos', 'los-conjuntos'),
    ('usando-a-matematica', 'aplicaciones-de-la-matematica'),
    ('conceitos-basicos-da-matematica', 'temas-basicos'),
    ('empreendedorismo', 'emprendimiento'),
    ('criatividade','creatividad'),
    ('os-caminhos-da-graduacao', 'acceso-a-la-educacion-superior'),
    ('credito',),
    ('conceitos-basicos-de-design-grafico', 'conceptos-basicos-de-diseno-grafico'),
]

In [16]:
to_translate = [('como-usar-o-telegram', 'curso-de-telegram'), ('google-sala-de-aula-para-alunos', 'google-classroom-para-estudiantes')]

In [17]:
for COURSE_PATH in to_translate:
    os.chdir(PATH + "/analisis_traducciones")
    if type(COURSE_PATH) is str:
        CURSO = COURSE_PATH
    else:
        CURSO = COURSE_PATH[0]

    new_dict = retrieve_lessons_text(db['lesson'], find_courses_by_name(collection_tutorials, COURSE_PATH))

    os.chdir('./data/translate/')

    if not os.path.isdir(CURSO):
        os.makedirs(CURSO)
    os.chdir(CURSO)

    for i in range(len(new_dict[CURSO]["es"])):
        if os.path.isfile('google_translate-' + CURSO + "-" + str(i) + ".txt"):
            continue
        else:
            with open('google_translate-' + CURSO + "-" + str(i) + ".txt","w") as f:
                f.write(translate_text(new_dict[CURSO]["es"][i]))

Found 1 tutorials for course como-usar-o-telegram 

Found 1 tutorials for course curso-de-telegram 

Found 1 tutorials for course google-sala-de-aula-para-alunos 

Found 1 tutorials for course google-classroom-para-estudiantes 



# Convert to Dataframe and Store as SQLiteDB

In [18]:
def listdirs(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [19]:
def listfiles(path):
    return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [ ]:
total_texts = []
for dir in listdirs(PATH + "/analisis_traducciones/data/translate"):
    os.chdir(PATH + "/analisis_traducciones/data/translate")
    os.chdir("./" + dir)
    files = listfiles(".")
    print(files)
    files.sort(key=(lambda x: int(x.split("-")[-1].split(".")[0])))
    translated_lessons = []
    for file in files:
        with open(file,"r") as f:
            translated_lessons.append(f.read())

    total_texts.append(translated_lessons)

    

In [ ]:
df = pd.DataFrame.from_dict(dict(zip(listdirs(PATH),total_texts)), orient='index')
df = df.transpose()

In [ ]:
df

,como-usar-o-sistema-android,criar-um-correio-eletronico,curso-de-photoshop,excel-2016,numeros-fracionarios,os-numeros-decimais,divisao,whatsapp-business,informatica-basica,conceitos-basicos-de-design-grafico,...,apresentacoes-basicas,conceitos-basicos-da-matematica,os-conjuntos-matematicos,criatividade,credito,como-usar-o-twitter,powerpoint-2019,usando-a-matematica,algebra,criacao-de-sites
0,Como excluir um aplicativo Para excluir um apl...,Como recuperar minha senha de e-mail do Gmail?...,Interface do Photoshop Antes de começar a trab...,Exportando um documento no Excel 2016 Por padr...,Frações Equivalentes O conceito de fração equi...,Decimais repetitivos puros para fração Decimai...,Divisão de um número Para aprender a fazer gra...,Como configurar as notificações do WhatsApp Bu...,Parte traseira da torre A maioria dos computad...,Teoria das Cores Essa teoria pode te ajudar em...,...,Teste básico de apresentações Você quer saber ...,Quiz Tópicos básicos de matemática Responda ao...,"Notação de conjuntos Como você acabou de ver, ...",Seguindo seus impulsos criativos Os impulsos p...,Teste de Crédito Responda as seguintes pergunt...,"Como tirar fotos, vídeos e transmitir ao vivo ...",Opção Régua A Opção Régua permite desenhar lin...,"Sistema Internacional de Unidades Sem dúvida, ...",Propriedades da inversa aditiva Saiba quais sã...,O que é um URL? Um site estático é composto po...
1,Opções de sincronização Você sabia que ao adic...,Baixar um anexo Você saberá se alguma de suas ...,Como personalizar a interface No Photoshop é m...,Imprimindo no Excel 2016 Às vezes não será suf...,Frações equivalentes a inteiros e naturais Voc...,Decimais recorrentes mistos para fração O méto...,Divisão de dois dígitos Agora que você sabe di...,O que são respostas rápidas no WhatsApp Busine...,Memória RAM e memória RAM do disco rígido É a ...,Como combinar as cores? Ao trabalhar com cores...,...,"Excluir slides Às vezes, você precisa excluir ...",O medo da matemática. Você já disse uma frase ...,O que é um conjunto? Para que você entenda o m...,Como ser um escritor? Ser escritor profissiona...,Recomendações para um empréstimo hipotecário A...,Como twittar com notas de voz? O Twitter lanço...,Teste o PowerPoint 2019 Descubra o quanto você...,Como Eratóstenes mediu a Terra? Talvez você já...,Inverso Multiplicativo Todo número tem dois op...,HTML - Hyper Text Markup Language HTML é uma l...
2,Como bloquear a tela no Android? Uma das medid...,Como recuperar minha senha de e-mail do Outloo...,Como criar um novo documento A criação de docu...,Como criar uma função no Excel Como vimos na p...,Rácios Os rácios são uma das ferramentas básic...,Decimais de teste Responda às seguintes pergun...,Quantas vezes? Saber quantas vezes uma quantid...,Como configurar respostas rápidas no iOS? Crie...,Placas de expansão A maioria dos computadores ...,Layout e composição Esses dois elementos são c...,...,Adicionando notas em slides As notas em suas a...,Anteriormente conhecemos a vida de um dos pers...,Classes de conjuntos Existem vários tipos de c...,Perfeccionismo no trabalho Embora você pense q...,O que você deve saber sobre a hipoteca Estes s...,Como postar e interagir no Twitter pelo celula...,Desenhar e escrever com uma caneta digital Des...,o que é um ano-luz? Você sabe a que distância ...,Como encontrar o valor absoluto de um número? ...,Sistemas de banco de dados Os bancos de dados ...
3,O que é e como usar um Widget? Widgets são mic...,Como recuperar minha senha de e-mail do Yahoo?...,"O que é Photoshop? Este é um programa, desenvo...",Aba Fórmulas no Excel 2016 Uma vez que a princ...,Simplificação de frações A simplificação de fr...,Problemas com divisão de decimais Quando se tr...,O que é dividir? A divisão é a operação que no...,Como configurar respostas rápidas no Android? ...,Qual é o teclado? O teclado é uma das principa...,Fundamentos do design São a base de todas as m...,...,Dicas para apresentações Aqui estão sete recom...,Já falamos sobre algar

# Escribir a SQLite DB

In [ ]:
import pandas as pd
import sqlite3 as sq

table_name = "translations"

conn = sq.connect('{}.sqlite'.format(table_name)) 
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

# Cargarlo de nuevo a df

In [ ]:
os.chdir("/Users/elefantickets/git/analisis_traducciones/")

In [ ]:
conn = sq.connect('{}.sqlite'.format(table_name))
df = pd.read_sql('select * from {}'.format(table_name), conn)
conn.close()

In [ ]:
df

,como-usar-o-sistema-android,criar-um-correio-eletronico,curso-de-photoshop,excel-2016,numeros-fracionarios,os-numeros-decimais,divisao,whatsapp-business,informatica-basica,conceitos-basicos-de-design-grafico,...,apresentacoes-basicas,conceitos-basicos-da-matematica,os-conjuntos-matematicos,criatividade,credito,como-usar-o-twitter,powerpoint-2019,usando-a-matematica,algebra,criacao-de-sites
0,Como excluir um aplicativo Para excluir um apl...,Como recuperar minha senha de e-mail do Gmail?...,Interface do Photoshop Antes de começar a trab...,Exportando um documento no Excel 2016 Por padr...,Frações Equivalentes O conceito de fração equi...,Decimais repetitivos puros para fração Decimai...,Divisão de um número Para aprender a fazer gra...,Como configurar as notificações do WhatsApp Bu...,Parte traseira da torre A maioria dos computad...,Teoria das Cores Essa teoria pode te ajudar em...,...,Teste básico de apresentações Você quer saber ...,Quiz Tópicos básicos de matemática Responda ao...,"Notação de conjuntos Como você acabou de ver, ...",Seguindo seus impulsos criativos Os impulsos p...,Teste de Crédito Responda as seguintes pergunt...,"Como tirar fotos, vídeos e transmitir ao vivo ...",Opção Régua A Opção Régua permite desenhar lin...,"Sistema Internacional de Unidades Sem dúvida, ...",Propriedades da inversa aditiva Saiba quais sã...,O que é um URL? Um site estático é composto po...
1,Opções de sincronização Você sabia que ao adic...,Baixar um anexo Você saberá se alguma de suas ...,Como personalizar a interface No Photoshop é m...,Imprimindo no Excel 2016 Às vezes não será suf...,Frações equivalentes a inteiros e naturais Voc...,Decimais recorrentes mistos para fração O méto...,Divisão de dois dígitos Agora que você sabe di...,O que são respostas rápidas no WhatsApp Busine...,Memória RAM e memória RAM do disco rígido É a ...,Como combinar as cores? Ao trabalhar com cores...,...,"Excluir slides Às vezes, você precisa excluir ...",O medo da matemática. Você já disse uma frase ...,O que é um conjunto? Para que você entenda o m...,Como ser um escritor? Ser escritor profissiona...,Recomendações para um empréstimo hipotecário A...,Como twittar com notas de voz? O Twitter lanço...,Teste o PowerPoint 2019 Descubra o quanto você...,Como Eratóstenes mediu a Terra? Talvez você já...,Inverso Multiplicativo Todo número tem dois op...,HTML - Hyper Text Markup Language HTML é uma l...
2,Como bloquear a tela no Android? Uma das medid...,Como recuperar minha senha de e-mail do Outloo...,Como criar um novo documento A criação de docu...,Como criar uma função no Excel Como vimos na p...,Rácios Os rácios são uma das ferramentas básic...,Decimais de teste Responda às seguintes pergun...,Quantas vezes? Saber quantas vezes uma quantid...,Como configurar respostas rápidas no iOS? Crie...,Placas de expansão A maioria dos computadores ...,Layout e composição Esses dois elementos são c...,...,Adicionando notas em slides As notas em suas a...,Anteriormente conhecemos a vida de um dos pers...,Classes de conjuntos Existem vários tipos de c...,Perfeccionismo no trabalho Embora você pense q...,O que você deve saber sobre a hipoteca Estes s...,Como postar e interagir no Twitter pelo celula...,Desenhar e escrever com uma caneta digital Des...,o que é um ano-luz? Você sabe a que distância ...,Como encontrar o valor absoluto de um número? ...,Sistemas de banco de dados Os bancos de dados ...
3,O que é e como usar um Widget? Widgets são mic...,Como recuperar minha senha de e-mail do Yahoo?...,"O que é Photoshop? Este é um programa, desenvo...",Aba Fórmulas no Excel 2016 Uma vez que a princ...,Simplificação de frações A simplificação de fr...,Problemas com divisão de decimais Quando se tr...,O que é dividir? A divisão é a operação que no...,Como configurar respostas rápidas no Android? ...,Qual é o teclado? O teclado é uma das principa...,Fundamentos do design São a base de todas as m...,...,Dicas para apresentações Aqui estão sete recom...,Já falamos sobre algar